In [1]:
import pandas as pd
import os
import collections 
import spacy
import pickle
from spacy.tokenizer import Tokenizer
import numpy as np
import pickle

In [ ]:
nlp = spacy.load('en')
tokenizer = Tokenizer(nlp.vocab)

filename = "full_articles.csv"
filepath = os.path.join(os.getcwd(), filename)
data = pd.read_csv(filepath)

data.dropna(how="any", subset=["title", "content", "publication"], inplace=True)

SEQ_LENGTH = 30
STEP = 1
SAVE_DIR = "pickles"

In [42]:
titles = data.content[:100].tolist()
contents = data.content[:100].tolist() # change to all later


titles = [title.lower() for title in titles] # change to all later
contents = [content.lower() for content in contents ] # change to all later

content_words = [word.text for doc in tokenizer.pipe(contents, batch_size=128) for word in doc]
title_words = [word.text for doc in tokenizer.pipe(titles, batch_size=128) for word in doc]

In [43]:
content_count = collections.Counter(content_words)
title_count = collections.Counter(title_words)

content_vocab = list(sorted([item[0] for item in content_count.most_common()]))
title_vocab = list(sorted([item[0] for item in title_count.most_common()]))


content_word_dict = {x: i for i,x in enumerate(content_vocab)}
title_word_dict = {x: i for i,x in enumerate(title_vocab)}

In [2]:
name = "content"
PKL_SAVE = f"{name}.pkl"
with open(os.path.join("pickles",PKL_SAVE), "wb") as pkl_file:
    pickle.dump((content_words, content_vocab, content_word_dict), pkl_file)
    
name = "title"
PKL_SAVE = f"{name}.pkl"
with open(os.path.join("pickles",PKL_SAVE), "wb") as pkl_file:
    pickle.dump((content_words, content_vocab, content_word_dict), pkl_file)

NameError: name 'content_words' is not defined

In [2]:
name = "content"
PKL_SAVE = f"{name}.pkl"
with open(os.path.join("pickles",PKL_SAVE), "rb") as pkl_file:
    content_words, content_vocab, content_word_dict = pickle.load(pkl_file)
    
#name = "title"
#PKL_SAVE = f"{name}.pkl"
#with open(os.path.join("pickles",PKL_SAVE), "rb") as pkl_file:
#    title_words, title_vocab, title_word_dict = pickle.load(pkl_file)
    
content_word_dict = {x: i for i,x in enumerate(content_vocab)}
#title_word_dict = {x: i for i,x in enumerate(title_vocab)}

In [3]:
SEQ_LENGTH = 5
STEP_SIZE = 1

sequences_content = []    
next_word_content = []


for i in range(0, len(content_words) - SEQ_LENGTH, STEP_SIZE):
    sequence = content_words[i:i+SEQ_LENGTH]
    next_word = content_words[i+SEQ_LENGTH]
    sequences_content.append(sequence)
    next_word_content.append(next_word)



In [ ]:
content_word_dict

In [4]:
# ONLY DOING content RN
# X = (num_sequences, SEQ_LENGTH, len(vocab))
# Y = (num_sequences, len_vocab)
n_sequences = len(sequences_content)
n_vocab = len(content_vocab)
X_content = np.zeros(shape=(n_sequences, SEQ_LENGTH, n_vocab), dtype=np.bool)
y_content = np.zeros(shape=(n_sequences, n_vocab), dtype=np.bool)

for i, sentence in enumerate(sequences_content):
    for t, word in enumerate(sentence):
        ind = content_word_dict[word]
        X_content[i, t, ind] = 1
    y_content[i, content_word_dict[next_word_content[i]]] = 1

In [5]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Bidirectional, LSTM, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy
n_nodes = 258
model = Sequential()
model.add(Bidirectional(LSTM(n_nodes, activation="relu"), input_shape = (SEQ_LENGTH, n_vocab)))
model.add(Dropout(0.6))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
optimizer = Adam(lr=0.001)
callbacks = [EarlyStopping(patience=2, monitor="val_loss")]
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=[categorical_accuracy])
MODEL_CHECK_DIR = "/checkpoints"
callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
           ModelCheckpoint(filepath=MODEL_CHECK_DIR + "/" + 'model_gen.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, mode='auto', period=2)]

batch_size = 16
epochs = 50

model.fit(X_content, y_content,
             batch_size = batch_size,
             epochs=epochs,
             shuffle=True,
              validation_split=0.1
             )

md.save(MODEL_CHECK_DIR + "/" + 'model_gen.h5')

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/paperspace/text-generation/env/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/paperspace/text-generation/env/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/paperspace/text-generation/env/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/paperspace/text-generation/env/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/paperspace/text-generation/env/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object fil